# Carbon fluxes validation 

Sea air carbon fluxes were calculated using the v2022 release of the Surface CO2 Atlas (https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.nodc:0160558). The Surface CO2 Atlas is a global data set of fCO2 (fugacity of carbon dioxide) in surface seawater and atmospheric pCO2 (partial pressure of carbon dioxide) with monthly resolution. The data set was developed by the NOAA National Centers for Environmental Information (NCEI) in collaboration with the University of Colorado (CU) and the NOAA Pacific Marine Environmental Laboratory (PMEL). The Surface CO2 Atlas is a gridded data set with a spatial resolution of 1° x 1° and a temporal resolution of one month. The data set covers the period from 1982 to 2021. The Surface CO2 Atlas is available as a netCDF file. The data set is updated annually. The Surface CO2 Atlas is a product of the NOAA Ocean Acidification Program (OAP) and the NOAA Ocean Observing and Monitoring Division (OOMD). The Surface CO2 Atlas is a contribution to the Global Ocean Data Analysis Project (GLODAP) and the Ocean Carbon Data System (OCADS). The Surface CO2 Atlas is a product of the NOAA Ocean Acidification Program (OAP) and the NOAA Ocean Observing and Monitoring Division (OOMD). The Surface CO2 Atlas is a contribution to the Global Ocean Data Analysis Project (GLODAP) and the Ocean Carbon Data System (OCADS).

The validation was carried out for the years with overlap between the model and observational time series. The model data was regridded to the observational grid using bilinear interpolation and the model data was averaged over the same time period as the observations. 

We used the variable "fgco2_smoothed" from the observational data.

In [ ]:
chunk_start
variable = "co2flux"

In [ ]:
ff = "../../matched/gridded/co2fluxes/co2fluxes.nc"
ds_model = nc.open_data(ff)
mask_all(ds_model)
years = set(ds_model.years)
year_min = min(years)
year_max = max(years)
year_range = f"{year_min}-{year_max}"
ds_model.subset(variable = "model")
ds_model.tmean("month")
ds_model.as_missing(0)
ds_model.run()
ds_annual = ds_model.copy()
ds_annual.tmean()
ds_annual.set_longnames({"model": "Sea surface carbon fluxes"})

In [ ]:
ds_obs = nc.open_data(ff)
ds_obs.subset(variable = "observation")
ds_obs.run()
ds_obs.tmean("month")
ds_obs.as_missing(0)
ds_obs.run()

ds_annual_obs = ds_obs.copy()
ds_annual_obs.tmean()
ds_annual_obs.set_longnames({"observation": "Sea surface carbon fluxes"})


In [ ]:

obs_mask = ds_obs.copy()
obs_mask > -1e20
mod_mask = ds_model.copy()
mod_mask > -1e20
mod_mask * obs_mask
mod_mask.run()
ds_model * mod_mask
ds_obs * mod_mask

In [ ]:
chunk_clim

In [ ]:
chunk_bias

The ability of the model to reproduce seasonality of carbon fluxes is assessed by comparing the modelled and observed seasonal cycle of carbon fluxes. The seasonal cycle is calculated by averaging the monthly values of carbon fluxes over all available model years. The seasonal cycle is calculated for each grid cell. The modelled seasonal cycle is compared to the observed seasonal cycle of carbon fluxes. The observed seasonal cycle is calculated by averaging the observed monthly values of carbon fluxes over all available years. The seasonal cycle is calculated for each grid cell. The modelled seasonal cycle is compared to the observed seasonal cycle using the correlation coefficient between the two. The correlation coefficient is calculated for each grid cell. The correlation coefficient ranges from -1 to 1. A value of 1 indicates a perfect agreement between the modelled and observed seasonal cycle of carbon fluxes. A value of -1 indicates a perfect disagreement between the modelled and observed seasonal cycle of carbon fluxes. A value of 0 indicates no agreement between the modelled and observed seasonal cycle of carbon fluxes.

In [ ]:
ds1 = ds_model.copy()
ds1.cdo_command("setname,model")
ds1.run()
ds2 = ds_obs.copy()
ds2.cdo_command("setname,observation")
ds2.run()
ds_cor = nc.open_data([ds1.current[0], ds2.current[0]])
ds_cor.merge(match=["month"])
ds_cor.run()
ds_ts = ds_cor.copy()
ds_cor.cor_time("model", "observation")
title = f"Seasonal temporal correlation between {variable} for model and observations"
ds_cor.run()

# output to nc

out = f"../../results/temporals/{variable}_cor.nc"
if not os.path.exists(os.path.dirname(out)):
    os.makedirs(os.path.dirname(out))
ds_cor.to_nc(out, zip = True, overwrite = True)

In [ ]:

df_cor = ds_cor.to_dataframe().reset_index()
# get range of lon and lat without missing values of cor in df_cor
plot_cor = ds_cor.pub_plot()

In [ ]:
md(f"**Figure {i_figure}**: Seasonal temporal correlation between model and observations for " + variable)
i_figure += 1

## Can the model reproduce long-term changes in sea-air CO2 fluxes? 

In [ ]:
md(f"The ability of the model to reproduce the observed long-term changes in {variable} is evaluated by calculating the linear trend of the time series. The trend is calculated for the years {year_range} using annual mean values.") 

In [ ]:
ds_lt = nc.open_data(ff)
ds_lt.tmean("year")
ds_lt.cor_time("model", "observation")
ds_lt.run()
plot_ts = ds_lt.pub_plot()

In [ ]:
md(f"**Figure {i_figure}**: The long-term temporal correlation between model and observations for surface carbon fluxes. The long-term correlation is calculated from {year_range} using annual means.")
i_figure += 1

In [ ]:
ds_global = nc.open_data(ff)
mask_all(ds_global)
ds_global.set_fill(-9999)
ds_global.tmean("year")
#
ds_mask = ds_global.copy()
ds_mask > -1e20
ds_mask.sum_all()
ds_mask.tmean()
ds_mask == 2
ds_mask.run()

ds_global * ds_mask
ds_global.run()
# ds_global
ds_global.run()
# 
ds_global.spatial_sum(by_area = True)
ds_global / 1e15
ds_global * 12
# 
df = (
    ds_global.to_dataframe().reset_index()
    .assign(year = lambda x: x.time_counter.dt.year)
    .loc[:,["year", "model", "observation"]]
    .melt(id_vars = "year", var_name = "source", value_name = "flux")
)
gg = (
    ggplot(df, aes(x = "year", y = "flux", color = "source")) +
    geom_line() +
    labs(y = "Oceanic carbon sink (GtC/yr)", x = "Year", color = "Source") +
    theme_bw()+
    theme(legend_position = "top")+
    labs(colour = "")

)
gg = gg.draw()
gg.show()


In [ ]:
md(f"**Figure {i_figure}**: Global oceanic carbon sink from model and observations. The sink is calculated from {year_range} using annual means.") 
i_figure += 1

In [ ]:
md_result , i_figure = ecoval.global_regionals(ds_model, ds_obs, variable, i_figure)

In [ ]:
md_result

In [ ]:

ds_annual = ds_model.copy()
ds_annual.rename({ds_annual.variables[0]: "model"})
ds_annual.append(ds_obs)
ds_annual.tmean()
ds_annual.merge("variables")
ds_annual.rename({ds_obs.variables[0]: "observation"})
out_dir = "../../results/annual_mean/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
out_file = out_dir + f"annualmean_{variable}.nc"
ds_annual.to_nc(out_file, zip = True, overwrite = True)



In [ ]:
chunk_end